## Autograd(자동 미분)
- `torch.autograd` 패키지는 Tensor의 모든 연산에 대해 **자동 미분** 제공
- 이는 코드를 어떻게 작성하여 실행하느냐에 따라 역전파가 정의된다는 뜻
- `backprop`을 위해 미분값을 자동으로 계산해줌


`requires_grad` 속성을 `True`로 설정하면, 해당 텐서에서 이루어지는 모든 연산들을 추적하기 시작 </br>
기록을 추적하는 것을 중단하게 하려면, `.detach()`를 호출하여 연산기록으로부터 분리

In [1]:
import torch

a = torch.randn(3, 3)
a = a * 3
print(a)
print(a.requires_grad) # 기본적으로 tensor는 requires_grad를 False로 가지게 된다.

tensor([[ 2.1762, -1.2215,  2.4816],
        [ 0.3373, -1.2746,  0.7586],
        [ 4.4176,  3.0812,  1.0303]])
False


`requires_grad_(...)`는 기존 텐서의 `requires_grad` 값을 바꿔치기 (`in-place`)하여 변경 </br>
`grad_fn` : 미분 값을 계산한 함수에 대한 정보 저장 (어떤 함수에 대해서 `backprop` 하였는지)

In [2]:
a.requires_grad_(True) # inplace 연산을 통해 바뀐 값을 바로 a에 넣어줌
print(a.requires_grad)

b = (a * a).sum()
print(b)
print(b.grad_fn) # gradient function, 즉 sum이라는 연산을 하였다.

True
tensor(44.7704, grad_fn=<SumBackward0>)


### 기울기 (Gradient)

In [3]:
x = torch.ones(3, 3, requires_grad=True)
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)


In [4]:
y = x + 5
print(y)

tensor([[6., 6., 6.],
        [6., 6., 6.],
        [6., 6., 6.]], grad_fn=<AddBackward0>)


In [6]:
z = y * y
out = z.mean()
print(z, out) # z는 multiplication 연산을 했고, out은 mean 연산이 수행되었다는 것을 알 수 있음

tensor([[36., 36., 36.],
        [36., 36., 36.],
        [36., 36., 36.]], grad_fn=<MulBackward0>) tensor(36., grad_fn=<MeanBackward0>)


계산이 완료된 후, `.backward()` 를 호출하면 자동으로 역전파 계산이 가능하고, `.grad` 속성에 누적됨

In [7]:
print(out)
out.backward() 

tensor(36., grad_fn=<MeanBackward0>)


`grad` : data가 거쳐온 layer에 대한 미분값들을 저장

In [8]:
print(x)
print(x.grad) # x는 모두 1로 되어있고, 이에 대한 미분 값을 볼 수 있음

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)
tensor([[1.3333, 1.3333, 1.3333],
        [1.3333, 1.3333, 1.3333],
        [1.3333, 1.3333, 1.3333]])


In [9]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2
print(y)

tensor([1129.5804, -822.6971,  694.0140], grad_fn=<MulBackward0>)


In [10]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


`with torch.no_grad()` 를 사용하여 기울기의 업데이트를 하지 않음 </br>
기록을 추적하는 것을 방지하기 위해 코드 블럭을 `with torch.no_grad()`로 감싸면 기울기 계산은 필요없지만, </br>
`requires_grad=True` 로 설정되어 학습 가능한 매개변수를 갖는 모델을 평가(evaluate) 할 때 유용함

In [11]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


`detach()` : 내용물(content)는 같지만 `requires_grad`가 다른 새로운 Tensor를 가져올 때

In [12]:
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all()) # x와 y의 값이 같은지

True
False
tensor(True)


### 자동 미분 흐름 예제
- 계산 흐름 $a \rightarrow b  \rightarrow c  \rightarrow out $

## $\quad \frac{\partial out}{\partial a} = ?$
- `backward()`를 통해 $a \leftarrow b  \leftarrow c  \leftarrow out $을 계산하면 $\frac{\partial out}{\partial a}$값이 `a.grad`에 채워짐

In [13]:
a = torch.ones(2, 2)
print(a)

tensor([[1., 1.],
        [1., 1.]])


In [14]:
a = torch.ones(2, 2, requires_grad=True)
print(a)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [15]:
print(a.data)
print(a.grad)
print(a.grad_fn)

tensor([[1., 1.],
        [1., 1.]])
None
None


### $b=a+2$

In [16]:
b = a + 2
print(b)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


### $c=b^2$

In [17]:
c = b ** 2
print(c)

tensor([[9., 9.],
        [9., 9.]], grad_fn=<PowBackward0>)


In [18]:
out = c.sum()
print(out)

tensor(36., grad_fn=<SumBackward0>)


In [19]:
print(out)
out.backward()

tensor(36., grad_fn=<SumBackward0>)


a의 `grad_fn` 이 None 인 이유는 직접적으로 계산한 부분이 없었기 때문

In [20]:
print(a.data)
print(a.grad)
print(a.grad_fn)

tensor([[1., 1.],
        [1., 1.]])
tensor([[6., 6.],
        [6., 6.]])
None


In [21]:
print(b.data)
print(b.grad)
print(b.grad_fn)

tensor([[3., 3.],
        [3., 3.]])
None


/home/hwan/anaconda3/envs/torch1.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  


In [22]:
print(c.data)
print(c.grad)
print(c.grad_fn)

tensor([[9., 9.],
        [9., 9.]])
None


/home/hwan/anaconda3/envs/torch1.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  


In [23]:
print(out.data)
print(out.grad)
print(out.grad_fn)

tensor(36.)
None


/home/hwan/anaconda3/envs/torch1.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  
